In [1]:
from ortools.linear_solver import pywraplp
from scipy.stats import poisson

# Create the solver
solver = pywraplp.Solver.CreateSolver('GLOP')
solver.SetTimeLimit(60000)  # Set time limit in milliseconds (60 seconds)

# Parameters
d = {0: 0, 1: 2, 2: 5}  # Period cost for each server state
K = 3  # Cost for changing the service rate
R = 5  # Reward for processing 1 job per period
S_max = 8  # Max system capacity

# Cost function for using server state b_prime when previous server state was b
def c(b, b_prime):
    return d[b_prime] if b == b_prime else d[b_prime] + 3

def h(s):
    return 2 * s

# Probability f_b(n) of processing n jobs if the service rate is b
def f(n, b):
    if b == 0:
        return 0.8 if n == 1 else 0.2 if n == 2 else 0.0
    elif b == 1:
        return 0.5 if n in {1, 2} else 0.0
    elif b == 2:
        return 0.2 if n == 1 else 0.8 if n == 2 else 0.0
    return 0.0

# Probability g(n) of n new jobs arriving
def g(n):
    return poisson.pmf(n, mu=1.5)

# Probability of n or more new jobs arriving
def g_or_more(n):
    return 1 - sum(g(m) for m in range(n))

# Sets of states (S: number of jobs in the system, B: server states)
S = range(S_max + 1)
B = [0, 1, 2]

# Define the expected reward R_{(s,b),a'} when state is (s,b) and chosen service rate is a
def r(s, b, a):
    if s == 0:
        return -c(b, a)
    elif s == 1:
        return R - h(s) - c(b, a)
    else:
        return R * sum(f(n, a) * n for n in [1, 2]) - h(s) - c(b, a)

# Transition probabilities
def P(s_i, b_i, s_j, b_j, a):
    if b_j != a:
        return 0.0
    if s_i == 0:
        return g(s_j) if s_j < S_max else g_or_more(s_j)
    elif s_i == 1:
        return g(s_j) if s_j < S_max else g_or_more(s_j)
    elif s_i <= S_max:
        if s_j < S_max:
            return sum(f(n, b_j) * g(s_j - s_i + n) for n in [1, 2])
        elif s_j == S_max:
            return sum(f(n, b_j) * g_or_more(s_j - s_i + n) for n in [1, 2])
    return 0.0

# Decision variables x_{(s,b),b'}: probability of choosing server state b' when state is (s,b)
x = {}
for s in S:
    for b in B:
        for b_prime in B:
            x[(s, b, b_prime)] = solver.NumVar(0.0, 1.0, f'x_{s}_{b}_{b_prime}')

# Initial state probabilities alpha
alpha = {}
n_total_states = len(S) * len(B)
for s in S:
    for b in B:
        alpha[(s, b)] = 1 / n_total_states

# Objective: maximize the expected reward
objective = solver.Objective()
for s in S:
    for b in B:
        for a in B:
            objective.SetCoefficient(x[(s, b, a)], r(s, b, a))
objective.SetMaximization()

# Balance constraints
for s_j in S:
    for b_j in B:
        # Right-hand side (outflow) - the sum of probabilities of choosing each action a in state (s_j, b_j)
        outflow = solver.Sum(x[(s_j, b_j, a)] for a in B)
        
        # Left-hand side (inflow) - the weighted sum of transition probabilities for each incoming state-action pair
        inflow = solver.Sum(
            P(s_i, b_i, s_j, b_j, a) * x[(s_i, b_i, a)]
            for s_i in S for b_i in B for a in B
        )
        
        # Add the constraint that the inflow and outflow must be equal
        solver.Add(inflow == outflow)

# Probability constraint to ensure all probabilities sum to 1
solver.Add(solver.Sum(x[(s, b, a)] for s in S for b in B for a in B) == 1)

# Solve the problem
status = solver.Solve()

# Print the optimal solution
if status == pywraplp.Solver.OPTIMAL:
    for s in S:
        print(f"If {s} jobs in the system:")
        for b in B:
            print(f"   If previous service rate was {b}:")
            for a in B:
                x_opt = x[(s, b, a)].solution_value()
                if x_opt > 0:
                    print(f"      Use service rate {a} (x={x_opt:.2f})")
else:
    print("No feasible solution found.")

If 0 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=0.06)
   If previous service rate was 1:
   If previous service rate was 2:
      Use service rate 0 (x=0.02)
If 1 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=0.10)
   If previous service rate was 1:
   If previous service rate was 2:
      Use service rate 0 (x=0.06)
If 2 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=0.09)
   If previous service rate was 1:
   If previous service rate was 2:
      Use service rate 2 (x=0.10)
If 3 jobs in the system:
   If previous service rate was 0:
      Use service rate 2 (x=0.05)
   If previous service rate was 1:
   If previous service rate was 2:
      Use service rate 2 (x=0.12)
If 4 jobs in the system:
   If previous service rate was 0:
      Use service rate 2 (x=0.02)
   If previous service rate was 1:
   If previous service rate was 2:
      Use service rate 2 (x=0.11)
If 5 jobs 